### Inputs

##### Basic information

* age

* sex
 * pregnant

* height
* weight

##### Symptoms

* fever
---
* fatigue
---
* sputum_production
---
* cough
 * dry
 * peristent
---
* smell_change
* taste_change
---
* shortness_of_breath
---
* chest_pain
---
* sore_throat
---
* headache
---
* myalgia
* arthralgia
---
* chills
---
* nausea
* vomiting
---
* runny_nose
* nasal_congestion
---
* diarrhea
* muscle_pain
 * abdominal_pain
---
* hemoptysis
---
* conjunctival_congestion
---
* brain_issue

##### Past medical history

* sinusitis_treatment
* antipretic_drugs (aspirin, ibuprufen, naproxen, paracetamol, ...)
* smoking (> 5 years)
* chronic_respiratory_disease
* hypertension
* cardiovalcular_disease
* diabete
 * poor_control_diabete
* chronic_renal_disease
* autoimmune_disease
* cancer
* organ_transplants
* hiv
* cytotoxic_drug
* immunosuppressive_drug

#### Biggest problem: A lot of data is available concerning positive tests and their symptoms, but it's difficult to estimate the statistics of negative cases.

Let's define S being the set of every possible symptom events. S<sub>i</sub> represents the event of contractig a certain symptom i.

U represent the person information (age, sex, medical_history, ...).

C represents the event correspondig to being positive to Coronavirus.

H represents the set of severity levels after contracting the Coronavirus.

---
The goal of our risk estimator is to compute P(C|Sx), with Sx a subset of S.

The goal of our severity estimator is to compute H(C,U,S) which can be directly computed from the data.

Most of data provide P(Sy|C), with Sy a subset of S.

---
We can consider having access to P(S|C) with values given from different existing reports. Then, to compute P(C|S) we will need P(C) and P(S<sub>i</sub>) ∀i, which are the variables of our model that we can adjust with data coming from complete data*. P(C) can be computed from Environment (E) model. 

H(C,U,S) is directly derived from the data statistics.

#### Model mainly based on Wuhan statistics
source: https://www.medrxiv.org/content/10.1101/2020.03.17.20037572v1

"Results: A total of 30 studies including 53000 patients with COVID-19 were included in this study, the mean age was 49.8 years (95% CI, 47.5-52.2 yrs) and 55.5% were male. The pooled incidence of severity and mortality were 20.2% (95% CI, 15.1-25.2%) and 3.1% (95% CI, 1.9-4.2%), respectively. The predictor for disease severity included old age (≥ 50 yrs, odds ratio [OR] = 2.61; 95% CI, 2.29-2.98), male (OR =1.348, 95% CI, 1.195-1.521), smoking (OR =1.734, 95% CI, 1.146-2.626) and any comorbidity (OR = 2.635, 95% CI, 2.098-3.309), especially chronic kidney disease (CKD, OR = 6.017; 95% CI, 2.192-16.514), chronic obstructive pulmonary disease (COPD, OR = 5.323; 95% CI, 2.613-10.847) and cerebrovascular disease (OR = 3.219; 95% CI, 1.486-6.972). In terms of laboratory results, increased lactate dehydrogenase (LDH), C-reactive protein (CRP) and D-dimer and decreased blood platelet and lymphocytes count were highly associated with severe COVID-19 (all for P < 0.001). Meanwhile, old age (≥ 60 yrs, RR = 9.45; 95% CI, 8.09-11.04), followed by cardiovascular disease (RR = 6.75; 95% CI, 5.40-8.43) hypertension (RR = 4.48; 95% CI, 3.69-5.45) and diabetes (RR = 4.43; 95% CI, 3.49-5.61) were found to be independent prognostic factors for the COVID-19 related death. Conclusions: To our knowledge, this is the first evidence-based medicine research to explore the risk factors of prognosis in patients with COVID-19, which is helpful to identify early-stage patients with poor prognosis and adapt effective treatment."

##### Statistics

WHO Report

Source: https://www.who.int/docs/default-source/coronaviruse/who-china-joint-mission-on-covid-19-final-report.pdf

In [1]:
who_stats = {
    'fever': 0.879,
    'fatigue': 0.381,
    'sputum_production': 0.334,
    'dry_cough': 0.677,
    'shortness_of_breath': 0.186,
    'sore_throat': 0.139,
    'headache': 0.136,
    'myalgia_arthralgia': 0.148,
    'chills': 0.114,
    'nausea': 0.05,
    'nasal_congestion': 0.48,
    'diarrhea': 0.37,
    'hemoptysis': 0.009,
    'conjunctival_congestion': 0.008
}

##### Prior

This is the variable part of our simple bayes model. Those values can be improved with the data that we get from data collection apps.

Those values have been invented

In [2]:
who_prior = {
    'fever': 2e-2,
    'fatigue': 5e-2,
    'sputum_production': 10e-3,
    'dry_cough': 10e-4,
    'shortness_of_breath': 5e-4,
    'sore_throat': 5e-2,
    'headache': 5e-2,
    'myalgia_arthralgia': 5e-2,
    'chills': 2e-3,
    'nausea': 10e-3,
    'nasal_congestion': 5e-2,
    'diarrhea': 5e-3,
    'hemoptysis': 2e-4,
    'conjunctival_congestion': 2e-2
}

In [3]:
covid_prior = 0.01

In [4]:
class Symptoms:
    def __init__(self, fever=False, fatigue=False, sputum_production=False, cough=False, dry_cough=False,
                 persistent_cough=False, smell_change=False, taste_change=False, shortness_of_breath=False,
                 breath_worsen=False, chest_pain=False, sore_throat=False, headache=False, myalgia=False,
                 arthralgia=False, chills=False, nausea=False, vomiting=False, runny_nose=False,
                 nasal_congestion=False, diarrhea=False, muscle_pain=False, abdominal_pain=False,
                 hemoptysis=False, conjunctival_congestion=False, brain_issue=False):
        
        self.fever = fever
        self.fatigue = fatigue
        self.sputum_production = sputum_production
        self.cough = cough or dry_cough or persistent_cough
        self.dry_cough = dry_cough
        self.persistent_cough = persistent_cough
        self.smell_change = smell_change
        self.taste_change = taste_change
        self.shortness_of_breath = shortness_of_breath or breath_worsen
        self.breath_worsen = breath_worsen
        self.chest_pain = chest_pain
        self.sore_throat = sore_throat
        self.headache = headache
        self.myalgia = myalgia
        self.arthralgia = arthralgia
        self.myalgia_arthralgia = myalgia or arthralgia
        self.chills = chills
        self.nausea = nausea
        self.vomiting = vomiting
        self.runny_nose = runny_nose
        self.nasal_congestion = nasal_congestion
        self.diarrhea = diarrhea
        self.muscle_pain = muscle_pain or abdominal_pain
        self.abdominal_pain = abdominal_pain
        self.hemoptysis = hemoptysis
        self.conjunctival_congestion = conjunctival_congestion
        self.brain_issue = brain_issue

In [5]:
class Person:
    def __init__(self, age, sex, height, weight, pregnant=False, symptoms=None, medical_history=None):
        self.age = age
        self.sex = sex
        self.height = height
        self.weight = weight
        self.pregnant = pregnant
        self.bmi = weight/(height*height)
        
        self.symptoms = Symptoms() if symptoms is None else symptoms
        self.medical_history = self.MedicalHistory() if medical_history is None else medical_history
            
    class MedicalHistory:
        def __init__(self, sinusitis_treatment=False, smoking=False, chronic_respiratory_disease=False,
                     hypertension=False, cardiovalcular_disease=False, diabete=False, poor_control_diabete=False,
                     chronic_renal_disease=False, autoimmune_disease=False, cancer=False, organ_transplants=False,
                     hiv=False, antipretic_drugs=False, cytotoxic_drug=False, immunosuppressive_drug=False):
            
            self.sinusitis_treatment=sinusitis_treatment
            self.smoking=smoking
            self.chronic_respiratory_disease=chronic_respiratory_disease
            self.hypertension=hypertension
            self.cardiovalcular_disease=cardiovalcular_disease
            self.diabete=diabete or poor_control_diabete
            self.poor_control_diabete=poor_control_diabete
            self.chronic_renal_disease=chronic_renal_disease
            self.autoimmune_disease=autoimmune_disease
            self.cancer=cancer
            self.organ_transplants=organ_transplants
            self.hiv=hiv
            self.antipretic_drugs=antipretic_drugs
            self.cytotoxic_drug=cytotoxic_drug
            self.immunosuppressive_drug=immunosuppressive_drug

In [6]:
def compute_risk(person):
    prob_no_covid = 1
    
    for symptom in who_stats:
        if(getattr(person.symptoms, symptom)):
            symptom_stat = who_stats[symptom]
            symptom_prior = who_prior[symptom]
            
            prob_no_covid_for_attribute = 1-(symptom_stat*covid_prior/symptom_prior)
            prob_no_covid = prob_no_covid*prob_no_covid_for_attribute
        
    return 1-prob_no_covid

Severity is a number between 0 and 3 which could also depend on the probability to have contracted the virus (to be discussed).

TODO: need to adjust the severity increasing according to better sources

In [7]:
max_severity = 3

In [8]:
def compute_severity(person, risk=None):
    severity = 0
    
    if(risk is None):
        risk = compute_risk(person)
        
    symptoms = person.symptoms
    medical_history = person.medical_history
    
    # High risk symptoms
    if(symptoms.chest_pain or symptoms.breath_worsen or symptoms.brain_issue):
        return max_severity
    
    # Other symptoms inducing severity
    if((symptoms.smell_change or symptoms.taste_change)
       and symptoms.headache and not person.medical_history.sinusitis_treatment):
        severity = severity + 1
    
    # Males
    if(person.sex == 'M'):
        severity = severity + 1
    
    # Obesity
    if(person.bmi > 40):
        severity = severity + 1
    
    # Age
    if(person.age > 50):
        severity = severity + 1
        
        if(person.age > 80):
            severity = severity + 1
    
    # Pregnancy
    if(person.pregnant):
        severity = severity + 1
     
    # Respiratory weaknesses
    if(medical_history.smoking or medical_history.chronic_respiratory_disease):
        severity = severity + 1
    
    # Autoimmune disabilities
    if(medical_history.autoimmune_disease or medical_history.hiv or medical_history.immunosuppressive_drug):
        severity = severity + 1
        
    # Drugs
    if(medical_history.antipretic_drugs or medical_history.cytotoxic_drug):
        severity = severity + 1
    
    if(medical_history.poor_control_diabete):
        severity = severity + 1
    
    if(medical_history.cancer):
        severity = severity + 1
    
    if(medical_history.cardiovalcular_disease or medical_history.hypertension):
        severity = severity + 1

    if(medical_history.chronic_renal_disease):
        severity = severity + 1
    
    if(medical_history.organ_transplants):
        severity = severity + 1    
    
    return min(severity, max_severity)

### Testing

In [9]:
person1 = Person(age=35, sex='M', height=1.86, weight=220)

person2 = Person(age=91, sex='M', height=1.86, weight=75)

person3 = Person(age=68, sex='M', height=1.86, weight=75)

person4 = Person(age=35, sex='M', height=1.86, weight=75)
person4.symptoms.chest_pain = True

person5 = Person(age=90, sex='M', height=1.86, weight=220)
person5.symptoms.chest_pain = True
person5.symptoms.fever = True

person6 = Person(age=90, sex='M', height=1.86, weight=220)
person6.symptoms.myalgia_arthralgia = True

person7 = Person(age=90, sex='M', height=1.86, weight=220)
person7.symptoms.myalgia_arthralgia = True
person7.symptoms.fatigue = True

person8 = Person(age=90, sex='M', height=1.86, weight=220)
person8.symptoms.myalgia_arthralgia = True
person8.symptoms.fatigue = True
person8.symptoms.chills = True
person8.symptoms.nosea = True
person8.symptoms.nasal_congestion = True

In [15]:
compute_severity(person5)

3

In [16]:
compute_risk(person5)

0.4395

In [17]:
compute_risk(person6)

0.02959999999999996

In [18]:
compute_risk(person7)

0.10354448000000005

In [19]:
compute_risk(person8)

0.6515298102656001